# Detecting Spam Messages

In this notebook we'll explore text message data and create models to predict if a message is spam or not. 

In [11]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [12]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)


### Task 1
What percentage of the documents in `spam_data` are spam?

*This function should return a float, the percent value (i.e. $ratio * 100$).*

In [13]:
def task_one():
    return spam_data[spam_data['target'] == 1]['target'].count()/spam_data['target'].count()*100

In [14]:
task_one()

13.406317300789663

### Task 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

def task_two():
    
    vect = CountVectorizer().fit(X_train)
    longest = 0
    string = ""
    for feature in vect.get_feature_names():
        if len(feature) > longest:
            string = feature
            longest = len(feature)
    return string

In [16]:
task_two()

'com1win150ppmx3age16subscription'

### Task 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [17]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def task_three():
    vect = CountVectorizer().fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    
    clfr = MultinomialNB(alpha=0.1)
    clfr.fit(X_train_vectorized, y_train)
    score = roc_auc_score(y_test, clfr.predict(vect.transform(X_test)))
    
    return score

In [18]:
task_three()

0.9720812182741116

### Task 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?

Put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. 

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

def task_four():
    vect = TfidfVectorizer().fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    feature_names = np.array(vect.get_feature_names())
    sorted_tfidf = np.sort(X_train_vectorized.max(0).toarray()[0])
    sorted_tfidf_index = X_train_vectorized.max(0).toarray()[0].argsort()
    
    out1 = pd.DataFrame({'col1': sorted_tfidf[:20], 'col2':  feature_names[sorted_tfidf_index[:20]]}).sort_values(by=['col1', 'col2'], ascending=[True, True])
    
    out2 = pd.DataFrame({'col1': sorted_tfidf[:-21:-1], 'col2': feature_names[sorted_tfidf_index[:-21:-1]]}).sort_values(by=['col1', 'col2'], ascending=[False, True])
    
    out1_ser = pd.Series(out1['col1'].values, index=out1['col2'])
    out2_ser = pd.Series(out2['col1'].values, index=out2['col2'])
    
    return out1_ser, out2_ser

In [20]:
task_four()

(col2
 aaniye          0.074475
 athletic        0.074475
 chef            0.074475
 companion       0.074475
 courageous      0.074475
 dependable      0.074475
 determined      0.074475
 exterminator    0.074475
 healer          0.074475
 listener        0.074475
 organizer       0.074475
 pest            0.074475
 psychiatrist    0.074475
 psychologist    0.074475
 pudunga         0.074475
 stylist         0.074475
 sympathetic     0.074475
 venaam          0.074475
 diwali          0.091250
 mornings        0.091250
 dtype: float64, col2
 146tf150p    1.000000
 645          1.000000
 anything     1.000000
 anytime      1.000000
 beerage      1.000000
 done         1.000000
 er           1.000000
 havent       1.000000
 home         1.000000
 lei          1.000000
 nite         1.000000
 ok           1.000000
 okie         1.000000
 thank        1.000000
 thanx        1.000000
 too          1.000000
 where        1.000000
 yup          1.000000
 tick         0.980166
 blank        0

### Task 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [21]:
def task_five():
    
    vect = TfidfVectorizer(min_df=3).fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    
    clfr = MultinomialNB(alpha=0.1)
    clfr.fit(X_train_vectorized, y_train)
    score = roc_auc_score(y_test, clfr.predict(vect.transform(X_test)))
    
    return score

In [22]:
task_five()

0.9416243654822335

### Task 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [23]:
def task_six():
    
    sum_spam = 0
    sum_not_spam = 0
    count_spam = 0
    count_not_spam = 0
    for index, row in spam_data.iterrows():
        if row['target'] == 1:
            sum_spam += len(row['text'])
            count_spam += 1
        else:
            sum_not_spam += len(row['text'])
            count_not_spam += 1
    return sum_not_spam/count_not_spam, sum_spam/count_spam
    
    

In [24]:
task_six()

(71.02362694300518, 138.8661311914324)

<br>
<br>
The following function has been provided to help combine new features into the training data:

In [25]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Task 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [26]:
from sklearn.svm import SVC

def len_of_doc():
    length_train = []
    length_test = []
    for text in X_train:
        length_train.append(len(text))
    for text in X_test:
        length_test.append(len(text))
    return length_train, length_test
    
def task_seven():
    
    length_train, length_test = len_of_doc()
    vect = TfidfVectorizer(min_df=5).fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    
    
    X_train_vectorized = add_feature(X_train_vectorized, length_train)
    clfr = SVC(C=10000).fit(X_train_vectorized, y_train)
    
    
    X_test_vectorized = add_feature(vect.transform(X_test), length_test)
    
    predicted = clfr.predict(X_test_vectorized)
    
    return roc_auc_score(y_test, predicted)

In [27]:
task_seven()

0.9581366823421557

### Task 8

What is the average number of digits per document for not spam and spam documents?

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [28]:
def task_eight():
    sum_spam = 0
    sum_not_spam = 0
    count_spam = 0
    count_not_spam = 0
    for index, row in spam_data.iterrows():
        if row['target'] == 1:
            for char in row['text']:
                if char.isdigit():
                    sum_spam += 1
            count_spam += 1
        else:
            for char in row['text']:
                if char.isdigit():
                    sum_not_spam += 1
            count_not_spam += 1
    return sum_not_spam/count_not_spam, sum_spam/count_spam
    
    

In [29]:
task_eight()

(0.2992746113989637, 15.759036144578314)

### Task 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [30]:
from sklearn.linear_model import LogisticRegression

def len_of_digits():
    digits_train = []
    digits_test = []
    
    for text in X_train:
        sum_spam = 0
        for char in text:
            if char.isdigit():
                sum_spam += 1
        digits_train.append(sum_spam)
        
    for text in X_test:
        sum_spam = 0
        for char in text:
            if char.isdigit():
                sum_spam += 1
        digits_test.append(sum_spam)
    
    return digits_train, digits_test
        

def task_nine():
    
    vect = TfidfVectorizer(min_df=5, ngram_range=(1,3)).fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    X_test_vectorized = vect.transform(X_test)
    
    
    length_train, length_test = len_of_doc()
    digits_train, digits_test = len_of_digits()
    
    X_train_vectorized = add_feature(X_train_vectorized, [length_train, digits_train])
    X_test_vectorized = add_feature(X_test_vectorized, [length_test, digits_test])
    
    clfr = LogisticRegression(C=100).fit(X_train_vectorized, y_train)
    predicted = clfr.predict(X_test_vectorized)
    
    return roc_auc_score(y_test, predicted)

In [31]:
task_nine()

0.9678709064054463

### Task 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [32]:
import re

def task_ten():
    sum_spam = 0
    sum_not_spam = 0
    count_spam = 0
    count_not_spam = 0
    for index, row in spam_data.iterrows():
        if row['target'] == 1:
            for char in row['text']:
                if re.match('\W', char):
                    sum_spam += 1
            count_spam += 1
        else:
            for char in row['text']:
                if re.match('\W', char):
                    sum_not_spam += 1
            count_not_spam += 1
    return sum_not_spam/count_not_spam, sum_spam/count_spam
    
    

In [33]:
task_ten()

(17.29181347150259, 29.041499330655956)

### Task 11

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [34]:
def len_of_non_word():
    non_word_train = []
    non_word_test = []
    
    for text in X_train:
        sum_spam = 0
        for char in text:
            if re.match('\W', char):
                sum_spam += 1
        non_word_train.append(sum_spam)
        
    for text in X_test:
        sum_spam = 0
        for char in text:
            if re.match('\W', char):
                sum_spam += 1
        non_word_test.append(sum_spam)
        
    return non_word_train, non_word_test

def task_eleven():
    
    vect = CountVectorizer(min_df=5, ngram_range=(2,5), analyzer='char_wb').fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    X_test_vectorized = vect.transform(X_test)
    
    length_train, length_test = len_of_doc()
    digits_train, digits_test = len_of_digits()
    non_word_train, non_word_test = len_of_non_word()
    
    X_train_vectorized = add_feature(X_train_vectorized, [length_train, digits_train, non_word_train])
    X_test_vectorized = add_feature(X_test_vectorized, [length_test, digits_test, non_word_test])
    
    clfr = LogisticRegression(C=100).fit(X_train_vectorized, y_train)
    predicted = clfr.predict(X_test_vectorized)
    
    #########
    
    feature_names = np.array(vect.get_feature_names())
    sorted_coef = np.sort(clfr.coef_[0])
    
    
    return roc_auc_score(y_test, predicted), sorted_coef[:10], sorted_coef[:-11:-1]
    

In [35]:
task_eleven()

(0.9788593110707434,
 array([-0.86974443, -0.86085162, -0.67695372, -0.66701678, -0.61488433,
        -0.57960044, -0.53506867, -0.50576055, -0.49851996, -0.49093339]),
 array([1.21220593, 0.59777064, 0.54149015, 0.53875326, 0.52146997,
        0.52036027, 0.51786555, 0.5160878 , 0.50866038, 0.50265377]))

### Achieved an AUC score of 0.9788593110707434